# SimMobility visualization - Congestion (traffic density)

Author: Lukas Ralf Schinzel, Master student at Technical University of Denmark (DTU)

**1. Information on Kepler GL Kupyter**

Documentation and installation of Kepler GL for Jupyter: https://docs.kepler.gl/docs/keplergl-jupyter  
User guide Kepler GL: https://docs.kepler.gl/docs/user-guides  
It is recommended to use Kepler GL for Jupyter in a Jupyter Notebook, not in Jupyter Lab.

**2. Visualization concept**

This visualization aims to compare two SimMobility scenarios with respect to congestion (traffic density) in the form of a linestring plot representing the road network. The coding of a link's stroke width is based on its traffic density. By means of selecting a temporal resolution, it is possible to choose between a static display (average traffic density during the day) and an animated display (development of the average traffic density over the 24 hours of a day). The dual-map view function of Kepler GL is used in order to be able to compare the traffic densities of two scenarios or of two different time points side by side.

The only input required by the user is to specify the input parameters in section 6. Other than that, it is only expected to rerun the notebook for each new visualization or when input parameters are changed.

The default configuration for the visualization in Kepler GL is saved in a Python script, which is automatically loaded from a configuration file when the visualization is created. The visualization can be adjusted using Kepler GL’s user interface.

**3. Required data**
- Link statistics
- Links (network)
- Configuration files

**4. Libraries**

In [1]:
# import libraries
import numpy as np
import pandas as pd
import datetime as dt
from shapely.geometry import Point, LineString
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
from keplergl import KeplerGl

**5. Functions**

In [2]:
def load_link_stats(filepath:str):
    
    '''Loads link statistics from CSV file.'''

    # define column names
    headers_link_stats = ['type', 'update_interval', 'id', 'length', 'density', 'no_vehicles_entered', 'no_vehicles_exited', 'no_cars_exited',
                          'no_taxis_exited', 'no_motorbikes_exited', 'no_buses_exited', 'no_others_exited']
    
    # define date types
    dtypes_link_stats = {'type': 'str', 'update_interval': 'int64', 'id': 'int64', 'length': 'float', 'density': 'float', 'no_vehicles_entered': 'float',
                         'no_vehicles_exited': 'float', 'no_cars_exited': 'float', 'no_taxis_exited': 'float', 'no_motorbikes_exited': 'float',
                         'no_buses_exited': 'float', 'no_others_exited': 'float'}
    
    # load link stats data
    df_link_stats = pd.read_csv(filepath, names=headers_link_stats, index_col=False, dtype = dtypes_link_stats,
                                usecols=['type', 'update_interval', 'id', 'density', 'no_vehicles_exited'])
    
    return df_link_stats

In [3]:
def get_link_densities(df_link_stats, df_links, aggregation='day'):
    
    '''Takes link statistics as input,
    calculates average densities for a given aggregation level (day or hour),
    and adds geometry information.'''
        
    # aggregate to day level
    if aggregation == 'day':
        df_link_densities = df_link_stats[['id','density']].groupby('id').mean()
        
    # aggregate to hour level
    elif aggregation == 'hour':
        df_link_densities = df_link_stats.copy()
        df_link_densities['update_interval'] = dt.datetime(1900,1,1) + pd.TimedeltaIndex(df_link_densities['update_interval'], unit='m') # define different date instead of 1.1.1900
        #df_link_densities['hour'] = df_link_densities.update_interval.dt.hour
        df_link_densities['hour'] = df_link_densities.update_interval.dt.floor('H')
        df_link_densities = df_link_densities.set_index('id')
        df_link_densities = df_link_densities.groupby(['id', 'hour']).mean()
    else:
        raise Exception("Please specify aggregation as either 'day' or 'hour'.")
    
    # join geometry data
    df_link_densities = df_link_densities.join(other=df_links['geometry'], on='id')
    df_link_densities = gpd.GeoDataFrame(df_link_densities, geometry='geometry')
    
    return df_link_densities

In [14]:
def load_links_from_csv(filepath:str, epsg):
    
    '''Loads links from CSV file.'''
    
    # load data from csv
    link_polyline = pd.read_csv(filepath)

    # combine x and y coordinates to shapely point
    link_polyline['point'] = [Point(xy) for xy in zip(link_polyline.x, link_polyline.y)]
    
    # initialize new dataframe
    links = pd.DataFrame(index=link_polyline.id.unique(), columns=['geometry'])

    # create linestrings and addd to initialized data frame
    for link_id in links.index: #np.arange(1,1000):#link_polyline.id:
        link_segments = link_polyline[link_polyline.id == link_id].sort_values('seq_id')
        links.loc[link_id,'geometry']=LineString(link_segments['point'].to_list())
        
    # convert to geojson
    df_links = gpd.GeoDataFrame(links, geometry='geometry')
    df_links = df_links.set_crs(epsg=str(epsg), allow_override=True)
    df_links = df_links.to_crs(epsg='4326') # necessary if correct CRS is set above?
    
    return df_links

**6. Specify input parameters**

Specify filepaths for data:

In [5]:
filepath_link_stats_A = '../Data/Singapore/AMOD/MH-BC/link_stats_file.csv'
filepath_link_stats_B = '../Data/Singapore/AMOD/MH-IN/link_stats_file.csv'
filepath_links = '../Data/Singapore/AMOD/simmobility_wgs84/link_polyline.csv'

Specify aggregation:
- 'day' for static visualization
- 'hour' for animated visualization of the 24 hours of the day

In [6]:
aggregation = 'day'

Specify EPSG in which links are stored:

In [7]:
epsg = '4326'

**7. Create visualization**

In [8]:
# load link statistics
df_link_stats_A = load_link_stats(filepath_link_stats_A)
df_link_stats_B = load_link_stats(filepath_link_stats_B)

In [9]:
# load links
df_links = load_links_from_csv(filepath_links, epsg=epsg)

In [10]:
# determine link densities
df_link_densities_A = get_link_densities(df_link_stats_A, df_links, aggregation=aggregation)
df_link_densities_B = get_link_densities(df_link_stats_B, df_links, aggregation=aggregation)

In [11]:
# create boxplot with link densities
if False:
    plt.figure(figsize=(25,2))
    plt.boxplot(df_link_densities_A.density, sym='.', vert=False)
    plt.xlabel('link density')
    plt.title('Boxplot link densities scenario A')
    plt.show()
    
    plt.figure(figsize=(25,2))
    plt.boxplot(df_link_densities_B.density, sym='.', vert=False)
    plt.xlabel('link density')
    plt.title('Boxplot link densities scenario B')
    plt.show()

In [12]:
# load the relevant map configuration saved
if aggregation == 'day':
    %run congestion_daily_scen_comparison_config.py # --> stores configuration in variable 'config'
elif aggregation =='hour':
    %run congestion_hourly_scen_comparison_config.py # --> stores configuration in variable 'config'
else:
    raise Exception("Please specify aggregation as either 'day' or 'hour'.")

In [16]:
# create visualization in KeplerGL
map_congestion = KeplerGl(height=800,
                          data={#'link_densities_A': df_link_densities_A[(df_link_densities_A.index.get_level_values('hour').hour < 10)].reset_index(), # restrict hours displayed if required
                                #'link_densities_B': df_link_densities_B[(df_link_densities_B.index.get_level_values('hour').hour < 10)].reset_index(), # restrict hours displayed if required
                                #'link_densities_A': df_link_densities_A[(df_link_densities_A.index.get_level_values('id') < 10000)].reset_index(), # restrict links displayed if required
                                #'link_densities_B': df_link_densities_B[(df_link_densities_B.index.get_level_values('id') < 10000)].reset_index(), # restrict links displayed if required
                                'link_densities_A': df_link_densities_A.reset_index(), # don't forget to reset index here!
                                'link_densities_B': df_link_densities_B.reset_index(), # don't forget to reset index here!
                               },
                           config=config,
                          )
map_congestion

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['link_densities_A'], 'id': '…

**8. Optional**

Saving new map configuration (optional):  
(mind that current configuration files will be overwritten if stored with same filename in same folder!)

Saving outputs to files (optional):